In [1]:
from __future__ import print_function, division

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, utils
import time
import os
import copy
import torch.nn as nn
import torch.nn.functional as F
import datetime

net_param_set = [{'conv':[(),
                          (3, 16, 3, 1, 0), 
                          (16, 32, 3, 1, 1),
                          (32, 32, 3, 1, 0),
                          (32, 16, 3, 1, 1)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             },
                 {'conv':[(),
                          (3, 16, 5, 1, 0), 
                          (16, 32, 3, 1, 1),
                          (32, 32, 3, 1, 0),
                          (32, 16, 3, 1, 1)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 60), 
                    (60, 10)] # in_channels, out_channels
             },
             {'conv':[(), 
                      (3, 32, 5, 1, 0), 
                      (32, 32, 3, 1, 0),
                      (32, 16, 3, 1, 1)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             },
             {'conv':[(), 
                      (3, 32, 5, 1, 0), 
                      (32, 16, 3, 1, 0)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             },
             {'conv':[(), 
                      (3, 32, 5, 1, 0), 
                      (32, 32, 5, 1, 0), 
                      (32, 16, 3, 1, 1)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*5*5, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             },
             {'conv':[(), 
                      (3, 32, 5, 1, 0), 
                      (32, 16, 5, 1, 0)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*5*5, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             }]
             
epoch_max = 40

optimizer_param_set = [(1e-5, 'expo'), 
                       (1e-4, 'expo'),
                       (5e-4, 'expo'),
                       (1e-5, 'step'), 
                       (1e-4, 'step'), 
                       (5e-4, 'step')] # learning rate, decay strategy 

In [2]:
def test(model):
    
    # overall training correct rate
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloaders['train'], 0):
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Training accuracy: %.2f %%' % (100 * correct / total))
    
    # overall testing correct rate
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloaders['test'], 0):
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Testing accuracy: %.2f %% (%d / %d)' % ((100 * correct / total), correct, total))
    
    # count testing predictions for each class
    correct_pred = {classname: 0 for classname in class_names}
    total_pred = {classname: 0 for classname in class_names}
    with torch.no_grad():
        for i, data in enumerate(dataloaders['test'], 0):
            images, labels = data
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[class_names[label]] += 1
                total_pred[class_names[label]] += 1

    # print accuracy for each class
    print("Testing accuracy (each class): ")
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print("{:1s}: {:.1f}%;  ".format(classname, accuracy), end=' ')
        if classname == "5":
            print()
    print()
    
    return


def train_test(model, criterion, optimizer, scheduler, num_epochs=25):
    
    for epoch in range(num_epochs):  

        running_loss = 0.0
        loss_record=[]
        for i, data in enumerate(dataloaders['train'], 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                loss_record.append(round(running_loss / 2000, 8))
                # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
                
        print(datetime.datetime.now(), ' Epoch', (epoch + 1), ': Average Loss', loss_record)

    print('Finished Training')
    
    test(model)
    
    return None

# Data transformer
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Dataset initialization
data_dir = 'data' 
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']} # Read train and test sets, respectively.

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) for x in ['train', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Set device to "cpu" if you have no gpu


In [3]:
class Net(nn.Module):
    """
    Input - 1x32x32
    Output - 10
    CONV1->CONV2->POOL1->CONV3->CONV4->POOL2->FC1->FC2->FC3
    """
    def __init__(self, params):
        super(Net, self).__init__()
        
        #Initialize layers
        self.params = params
        self.n_layers = len(params['conv'])+len(params['pool'])+len(params['fc'])-3
        self.printed = False
        
        if self.n_layers == 9:
            self.conv1 = nn.Conv2d(*self.params['conv'][1])
            self.conv2 = nn.Conv2d(*self.params['conv'][2])
            self.conv3 = nn.Conv2d(*self.params['conv'][3])
            self.conv4 = nn.Conv2d(*self.params['conv'][4])
            
            self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
            self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
            
            self.fc1 = nn.Linear(*self.params['fc'][1])
            self.fc2 = nn.Linear(*self.params['fc'][2])
            self.fc3 = nn.Linear(*self.params['fc'][3])
            
        elif self.n_layers == 8:
            self.conv1 = nn.Conv2d(*self.params['conv'][1])
            self.conv2 = nn.Conv2d(*self.params['conv'][2])
            self.conv3 = nn.Conv2d(*self.params['conv'][3])
            
            self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
            self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
            
            self.fc1 = nn.Linear(*self.params['fc'][1])
            self.fc2 = nn.Linear(*self.params['fc'][2])
            self.fc3 = nn.Linear(*self.params['fc'][3])
        
        elif self.n_layers == 7:
            self.conv1 = nn.Conv2d(*self.params['conv'][1])
            self.conv2 = nn.Conv2d(*self.params['conv'][2])
            
            self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
            self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
            
            self.fc1 = nn.Linear(*self.params['fc'][1])
            self.fc2 = nn.Linear(*self.params['fc'][2])
            self.fc3 = nn.Linear(*self.params['fc'][3])
            

    def forward(self, img):
        # Implement forward pass
        x = img
        
        if self.n_layers == 9:
            x = F.relu(self.conv1(x))
            if not self.printed: 
                print("CONV1", x.size(), end=" || ")
            x = F.relu(self.conv2(x))
            if not self.printed: 
                print("CONV2", x.size(), end=" || ")
            x = self.pool1(x)
            if not self.printed: 
                print("POOL1", x.size())
        
            x = F.relu(self.conv3(x))
            if not self.printed: 
                print("CONV3", x.size(), end=" || ")
            x = F.relu(self.conv4(x))
            if not self.printed: 
                print("CONV4", x.size(), end=" || ")
            x = self.pool2(x)
            if not self.printed: 
                print("POOL2", x.size())
        
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            if not self.printed: 
                print("FC1", x.size(), end=" || ")
            x = F.relu(self.fc2(x))
            if not self.printed: 
                print("FC2", x.size(), end=" || ")
            x = self.fc3(x)
            if not self.printed: 
                print("FC3", x.size())
                
        elif self.n_layers == 8:
            x = F.relu(self.conv1(x))
            if not self.printed: 
                print("CONV1", x.size(), end=" || ")
            x = self.pool1(x)
            if not self.printed: 
                print("POOL1", x.size())
        
            x = F.relu(self.conv2(x))
            if not self.printed: 
                print("CONV2", x.size(), end=" || ")
            x = F.relu(self.conv3(x))
            if not self.printed: 
                print("CONV3", x.size(), end=" || ")
            x = self.pool2(x)
            if not self.printed: 
                print("POOL2", x.size())
        
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            if not self.printed: 
                print("FC1", x.size(), end=" || ")
            x = F.relu(self.fc2(x))
            if not self.printed: 
                print("FC2", x.size(), end=" || ")
            x = self.fc3(x)
            if not self.printed: 
                print("FC3", x.size())
        
        elif self.n_layers == 7:
            x = F.relu(self.conv1(x))
            if not self.printed: 
                print("CONV1", x.size(), end=" || ")
            x = self.pool1(x)
            if not self.printed: 
                print("POOL1", x.size())
        
            x = F.relu(self.conv2(x))
            if not self.printed: 
                print("CONV2", x.size(), end=" || ")
            x = self.pool2(x)
            if not self.printed: 
                print("POOL2", x.size())
        
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            if not self.printed: 
                print("FC1", x.size(), end=" || ")
            x = F.relu(self.fc2(x))
            if not self.printed: 
                print("FC2", x.size(), end=" || ")
            x = self.fc3(x)
            if not self.printed: 
                print("FC3", x.size())
        
        self.printed = True

        return x


In [5]:
paths = []
for j in range(len(optimizer_param_set)):
    for i in range(len(net_param_set)):
        model_ft = Net(net_param_set[i])
        model_ft = model_ft.to(device)
        criterion = nn.CrossEntropyLoss()
        learning_rate, decay_strategy = optimizer_param_set[j]
        optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)
        if decay_strategy=='expo':
            lr_scheduler_ft = lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.1)
        elif decay_strategy=='step':
            lr_scheduler_ft = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)
        else:
            print("ERROR 2")
        
        print(i, ",", j)
        for n in range((epoch_max // 5)):
            print(datetime.datetime.now())
            epo = 5*n+5
            print("epoch range: ", epo-4, " to ", epo)
            train_test(model_ft, criterion, optimizer_ft, lr_scheduler_ft, num_epochs=5)
        
        PATH = "./cnn"+str(i)+"_"+str(j)+"_.pth"
        paths.append(PATH)
        torch.save({'epoch': epoch_max,
                    'model_state_dict': model_ft.state_dict(),
                    'optimizer_state_dict': optimizer_ft.state_dict()
                   }, PATH)
        
        print()


0 , 0
2021-05-09 04:47:40.660795
epoch range:  1  to  5
CONV1 torch.Size([4, 16, 28, 28]) || CONV2 torch.Size([4, 32, 28, 28]) || POOL1 torch.Size([4, 32, 14, 14])
CONV3 torch.Size([4, 32, 12, 12]) || CONV4 torch.Size([4, 16, 12, 12]) || POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 90]) || FC3 torch.Size([4, 10])
2021-05-09 04:48:32.807610  Epoch 1 : Average Loss [2.30353685, 2.30253172, 2.30099708]
2021-05-09 04:49:17.542386  Epoch 2 : Average Loss [2.28496971, 2.24931119, 2.17239756]
2021-05-09 04:50:06.541664  Epoch 3 : Average Loss [1.95230017, 1.81345538, 1.71424484]
2021-05-09 04:50:56.668443  Epoch 4 : Average Loss [1.59188034, 1.53490699, 1.49808787]
2021-05-09 04:51:48.400552  Epoch 5 : Average Loss [1.46116918, 1.41545225, 1.40268621]
Finished Training
Training accuracy: 54.16 %
Testing accuracy: 55.40 % (2770 / 5000)
Testing accuracy (each class): 
0: 74.2%;   1: 70.0%;   2: 51.8%;   3: 26.6%;   4: 73.4%;   5: 47.4%;   
6: 45.4%;   7: 71.2%;

2021-05-09 05:37:25.541903  Epoch 4 : Average Loss [1.1590099, 1.1630903, 1.1519386]
2021-05-09 05:38:15.851106  Epoch 5 : Average Loss [1.12520948, 1.12606154, 1.11223769]
Finished Training
Training accuracy: 65.03 %
Testing accuracy: 66.36 % (3318 / 5000)
Testing accuracy (each class): 
0: 73.2%;   1: 75.0%;   2: 64.0%;   3: 60.6%;   4: 78.2%;   5: 59.0%;   
6: 54.2%;   7: 81.4%;   8: 53.0%;   9: 65.0%;   
2021-05-09 05:38:46.050390
epoch range:  16  to  20
2021-05-09 05:39:34.051231  Epoch 1 : Average Loss [1.10578728, 1.08578116, 1.06617404]
2021-05-09 05:40:24.533954  Epoch 2 : Average Loss [1.07495935, 1.04803041, 1.04168052]
2021-05-09 05:41:15.499066  Epoch 3 : Average Loss [1.04416911, 1.0401398, 1.00895645]
2021-05-09 05:42:06.463072  Epoch 4 : Average Loss [0.99762966, 1.01084458, 1.0037293]
2021-05-09 05:42:56.760710  Epoch 5 : Average Loss [0.99047811, 0.9601208, 0.96361329]
Finished Training
Training accuracy: 68.97 %
Testing accuracy: 68.64 % (3432 / 5000)
Testing accura

2021-05-09 06:24:55.765750  Epoch 3 : Average Loss [0.68849027, 0.68116953, 0.6889324]
2021-05-09 06:25:38.795427  Epoch 4 : Average Loss [0.66478517, 0.68187249, 0.68171416]
2021-05-09 06:26:21.918806  Epoch 5 : Average Loss [0.66894343, 0.65125645, 0.67260772]
Finished Training
Training accuracy: 80.39 %
Testing accuracy: 78.28 % (3914 / 5000)
Testing accuracy (each class): 
0: 83.0%;   1: 82.8%;   2: 79.8%;   3: 64.2%;   4: 84.2%;   5: 75.8%;   
6: 72.2%;   7: 87.6%;   8: 71.0%;   9: 82.2%;   
2021-05-09 06:26:50.234011
epoch range:  31  to  35
2021-05-09 06:27:31.478896  Epoch 1 : Average Loss [0.638271, 0.65617979, 0.66530028]
2021-05-09 06:28:14.308105  Epoch 2 : Average Loss [0.64402735, 0.64148446, 0.64371931]
2021-05-09 06:28:56.937520  Epoch 3 : Average Loss [0.64677608, 0.62593295, 0.63848221]
2021-05-09 06:29:40.440409  Epoch 4 : Average Loss [0.62648309, 0.63449215, 0.62587058]
2021-05-09 06:30:23.342774  Epoch 5 : Average Loss [0.62977259, 0.61297012, 0.61997414]
Finished

2021-05-09 07:03:46.504254  Epoch 1 : Average Loss [2.30427377, 2.30321988, 2.30005986]
2021-05-09 07:04:35.563588  Epoch 2 : Average Loss [2.26358038, 2.15478753, 1.98915722]
2021-05-09 07:05:23.080956  Epoch 3 : Average Loss [1.75760043, 1.67808943, 1.6250682]
2021-05-09 07:06:10.900930  Epoch 4 : Average Loss [1.56948492, 1.5267308, 1.51853952]
2021-05-09 07:06:58.386648  Epoch 5 : Average Loss [1.47629603, 1.44903782, 1.43781036]
Finished Training
Training accuracy: 53.36 %
Testing accuracy: 52.64 % (2632 / 5000)
Testing accuracy (each class): 
0: 67.2%;   1: 68.6%;   2: 56.6%;   3: 36.0%;   4: 63.0%;   5: 46.0%;   
6: 29.6%;   7: 74.6%;   8: 49.2%;   9: 35.6%;   
2021-05-09 07:07:26.819111
epoch range:  6  to  10
2021-05-09 07:08:14.415002  Epoch 1 : Average Loss [1.39284111, 1.4119969, 1.3411594]
2021-05-09 07:09:01.428255  Epoch 2 : Average Loss [1.304267, 1.31838167, 1.30194416]
2021-05-09 07:09:47.895316  Epoch 3 : Average Loss [1.23768727, 1.20432521, 1.21566188]
2021-05-09 0

2021-05-09 07:50:34.759390  Epoch 1 : Average Loss [0.82285092, 0.84309857, 0.84081489]
2021-05-09 07:51:20.255894  Epoch 2 : Average Loss [0.81617447, 0.8160796, 0.82689809]
2021-05-09 07:52:05.554660  Epoch 3 : Average Loss [0.81395271, 0.79383504, 0.80190351]
2021-05-09 07:52:50.511334  Epoch 4 : Average Loss [0.80097021, 0.77677839, 0.79400275]
2021-05-09 07:53:35.533792  Epoch 5 : Average Loss [0.75938516, 0.78785332, 0.75761592]
Finished Training
Training accuracy: 77.35 %
Testing accuracy: 76.10 % (3805 / 5000)
Testing accuracy (each class): 
0: 83.2%;   1: 84.2%;   2: 82.4%;   3: 62.6%;   4: 83.2%;   5: 61.8%;   
6: 64.6%;   7: 85.0%;   8: 74.2%;   9: 79.8%;   
2021-05-09 07:54:02.505550
epoch range:  21  to  25
2021-05-09 07:54:47.043346  Epoch 1 : Average Loss [0.75529389, 0.74233567, 0.76868381]
2021-05-09 07:55:31.759775  Epoch 2 : Average Loss [0.74692954, 0.74240699, 0.74578639]
2021-05-09 07:56:16.568411  Epoch 3 : Average Loss [0.71991663, 0.72365829, 0.74732933]
2021-0

...

2021-05-09 12:48:24.525951  Epoch 1 : Average Loss [0.21818604, 0.254143, 0.23994652]
2021-05-09 12:49:18.657877  Epoch 2 : Average Loss [0.23264532, 0.22950177, 0.23535206]
2021-05-09 12:50:13.419110  Epoch 3 : Average Loss [0.21592085, 0.20706516, 0.22307861]
2021-05-09 12:51:07.766534  Epoch 4 : Average Loss [0.1943048, 0.22296854, 0.22627181]
2021-05-09 12:52:11.903197  Epoch 5 : Average Loss [0.20921124, 0.21614683, 0.20457886]
Finished Training
Training accuracy: 93.92 %
Testing accuracy: 86.38 % (4319 / 5000)
Testing accuracy (each class): 
0: 84.8%;   1: 84.4%;   2: 88.2%;   3: 78.6%;   4: 89.4%;   5: 87.2%;   
6: 86.6%;   7: 91.6%;   8: 84.6%;   9: 88.4%;   
2021-05-09 12:52:46.972584
epoch range:  26  to  30
2021-05-09 12:53:41.089875  Epoch 1 : Average Loss [0.19389058, 0.21315825, 0.21630416]
2021-05-09 12:54:32.192446  Epoch 2 : Average Loss [0.18543252, 0.21752928, 0.2111706]
2021-05-09 12:55:26.387614  Epoch 3 : Average Loss [0.18039285, 0.21511282, 0.21211516]
2021-05-0

2021-05-09 13:36:16.589730  Epoch 1 : Average Loss [0.15791373, 0.17690343, 0.18072406]
2021-05-09 13:36:58.554609  Epoch 2 : Average Loss [0.15137143, 0.17115883, 0.18933294]
2021-05-09 13:37:39.042131  Epoch 3 : Average Loss [0.16571774, 0.16284598, 0.18126745]
2021-05-09 13:38:21.977560  Epoch 4 : Average Loss [0.14889964, 0.18309045, 0.16615683]
2021-05-09 13:39:07.837341  Epoch 5 : Average Loss [0.16718459, 0.15554754, 0.17454968]
Finished Training
Training accuracy: 95.58 %
Testing accuracy: 84.60 % (4230 / 5000)
Testing accuracy (each class): 
0: 87.8%;   1: 81.6%;   2: 87.4%;   3: 80.2%;   4: 89.2%;   5: 82.0%;   
6: 80.8%;   7: 88.4%;   8: 85.2%;   9: 83.4%;   

3 , 2
2021-05-09 13:39:32.702512
epoch range:  1  to  5
CONV1 torch.Size([4, 32, 28, 28]) || POOL1 torch.Size([4, 32, 14, 14])
CONV2 torch.Size([4, 16, 12, 12]) || POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 90]) || FC3 torch.Size([4, 10])
2021-05-09 13:40:05.443278  Epoch 1 : Average

Training accuracy: 88.52 %
Testing accuracy: 84.44 % (4222 / 5000)
Testing accuracy (each class): 
0: 83.4%;   1: 87.0%;   2: 78.6%;   3: 76.6%;   4: 83.4%;   5: 86.0%;   
6: 85.0%;   7: 88.0%;   8: 88.2%;   9: 88.2%;   
2021-05-09 14:15:10.440407
epoch range:  11  to  15
2021-05-09 14:16:00.367415  Epoch 1 : Average Loss [0.3674395, 0.37303381, 0.37606535]
2021-05-09 14:16:50.423484  Epoch 2 : Average Loss [0.34407149, 0.34701969, 0.38251926]
2021-05-09 14:17:36.093037  Epoch 3 : Average Loss [0.33987825, 0.34466968, 0.35681078]
2021-05-09 14:18:24.700586  Epoch 4 : Average Loss [0.32321055, 0.34426279, 0.35080234]
2021-05-09 14:19:16.127836  Epoch 5 : Average Loss [0.31916876, 0.33456313, 0.33671503]
Finished Training
Training accuracy: 89.99 %
Testing accuracy: 84.94 % (4247 / 5000)
Testing accuracy (each class): 
0: 84.8%;   1: 83.8%;   2: 85.0%;   3: 77.6%;   4: 92.0%;   5: 81.4%;   
6: 80.2%;   7: 91.8%;   8: 87.4%;   9: 85.4%;   
2021-05-09 14:19:40.426323
epoch range:  16  to  

Training accuracy: 89.61 %
Testing accuracy: 84.48 % (4224 / 5000)
Testing accuracy (each class): 
0: 86.4%;   1: 86.2%;   2: 84.2%;   3: 80.0%;   4: 90.6%;   5: 86.6%;   
6: 79.0%;   7: 87.0%;   8: 79.6%;   9: 85.2%;   
2021-05-09 15:02:50.967692
epoch range:  26  to  30
2021-05-09 15:03:35.772828  Epoch 1 : Average Loss [0.31435009, 0.31655268, 0.31167327]
2021-05-09 15:04:23.031086  Epoch 2 : Average Loss [0.29400186, 0.32753986, 0.31609962]
2021-05-09 15:05:05.114294  Epoch 3 : Average Loss [0.28097659, 0.32002451, 0.32340104]
2021-05-09 15:05:47.627145  Epoch 4 : Average Loss [0.28706184, 0.31190966, 0.30250702]
2021-05-09 15:06:32.294333  Epoch 5 : Average Loss [0.28866431, 0.30657516, 0.29920571]
Finished Training
Training accuracy: 89.27 %
Testing accuracy: 83.28 % (4164 / 5000)
Testing accuracy (each class): 
0: 85.0%;   1: 88.6%;   2: 90.8%;   3: 75.2%;   4: 86.8%;   5: 82.8%;   
6: 77.8%;   7: 88.0%;   8: 84.4%;   9: 73.4%;   
2021-05-09 15:06:57.868941
epoch range:  31  to 

KeyboardInterrupt: 

In [7]:
for i in range(len(paths)):
    PATH = paths[i]
    print(PATH)
    
    j = int(PATH[5]) # net parameter
    k = int(PATH[7]) # optimizer parameter
    model = Net(net_param_set[j])
    learning_rate, decay_strategy = optimizer_param_set[k]
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    
    model.eval()
    test(model)
    print()


./cnn0_0_.pth
CONV1 torch.Size([4, 16, 28, 28]) || CONV2 torch.Size([4, 32, 28, 28]) || POOL1 torch.Size([4, 32, 14, 14])
CONV3 torch.Size([4, 32, 12, 12]) || CONV4 torch.Size([4, 16, 12, 12]) || POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 90]) || FC3 torch.Size([4, 10])
Training accuracy: 82.07 %
Testing accuracy: 79.78 % (3989 / 5000)
Testing accuracy (each class): 
0: 83.2%;   1: 81.8%;   2: 77.0%;   3: 71.8%;   4: 86.2%;   5: 72.2%;   
6: 78.8%;   7: 87.6%;   8: 78.0%;   9: 81.2%;   

./cnn1_0_.pth
CONV1 torch.Size([4, 16, 28, 28]) || CONV2 torch.Size([4, 32, 28, 28]) || POOL1 torch.Size([4, 32, 14, 14])
CONV3 torch.Size([4, 32, 12, 12]) || CONV4 torch.Size([4, 16, 12, 12]) || POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 60]) || FC3 torch.Size([4, 10])
Training accuracy: 81.27 %
Testing accuracy: 79.40 % (3970 / 5000)
Testing accuracy (each class): 
0: 88.0%;   1: 81.8%;   2: 78.4%;   3: 64.2%;   4: 84.4%;   5: 75

Testing accuracy: 84.80 % (4240 / 5000)
Testing accuracy (each class): 
0: 88.2%;   1: 90.6%;   2: 81.4%;   3: 75.4%;   4: 88.6%;   5: 87.6%;   
6: 83.4%;   7: 88.4%;   8: 83.0%;   9: 81.4%;   

./cnn5_2_.pth
CONV1 torch.Size([4, 32, 28, 28]) || POOL1 torch.Size([4, 32, 14, 14])
CONV2 torch.Size([4, 16, 10, 10]) || POOL2 torch.Size([4, 16, 5, 5])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 90]) || FC3 torch.Size([4, 10])
Training accuracy: 91.04 %
Testing accuracy: 83.98 % (4199 / 5000)
Testing accuracy (each class): 
0: 87.2%;   1: 80.6%;   2: 81.6%;   3: 76.4%;   4: 89.4%;   5: 81.4%;   
6: 83.2%;   7: 88.2%;   8: 84.6%;   9: 87.2%;   



In [6]:
paths

['./cnn0_0_.pth',
 './cnn1_0_.pth',
 './cnn2_0_.pth',
 './cnn3_0_.pth',
 './cnn4_0_.pth',
 './cnn5_0_.pth',
 './cnn0_1_.pth',
 './cnn1_1_.pth',
 './cnn2_1_.pth',
 './cnn3_1_.pth',
 './cnn4_1_.pth',
 './cnn5_1_.pth',
 './cnn0_2_.pth',
 './cnn1_2_.pth',
 './cnn2_2_.pth',
 './cnn3_2_.pth',
 './cnn4_2_.pth',
 './cnn5_2_.pth']